In [1]:
import pandas as pd
import numpy as np
import branca.colormap as cm  # Used for color gradient
import folium
import geopandas

In [2]:
df_sensors = pd.read_csv(r"C:\Users\rueed\OneDrive\HSLU\3 Semester\DSPRO 1\HSLU_DSPRO1_TrafficStatus\data\RawDataLondon\London_detectors.csv")
df_anomalies = pd.read_csv(r"C:\Users\rueed\OneDrive\HSLU\3 Semester\DSPRO 1\data\Anomalies_31.10.2024.csv")


In [3]:
df_sensors.head()

,detid,length,pos,fclass,road,limit,citycode,lanes,linkid,long,lat
0,EAST_N04/161x1,0.303585,0.261157,secondary,Homerton Road,NaN,london,1.0,5082.0,-0.021497,51.550929
1,EAST_N04/161y1,0.103679,0.063417,primary,Eastway,NaN,london,1.0,5091.0,-0.020899,51.550704
2,EAST_N04/162a1,0.260623,0.117906,secondary,Homerton Road,NaN,london,1.0,5083.0,-0.022649,51.550907
3,EAST_N04/162a2,0.216874,0.117942,secondary,Homerton Road,NaN,london,1.0,5084.0,-0.022617,51.550880
4,EAST_N04/163f1,0.344754,0.329789,primary,Eastway,NaN,london,1.0,5092.0,-0.019288,51.552281


In [4]:
df_anomalies

,detid,IQR_out_of_bound,IQR_to_small,not_enough_data
0,CNTR_N00/005b1,False,False,True
1,CNTR_N00/005g1,False,False,True
2,CNTR_N00/005g2,False,False,True
3,CNTR_N00/005x1,False,False,True
4,CNTR_N00/005x2,False,False,True
...,...,...,...,...
3535,SOUT_N32/999a4,False,True,False
3536,SOUT_N32/999b1,False,True,True
3537,SOUT_N32/999b2,False,True,False
3538,SOUT_N32/999b3,False,True,False


In [22]:
df_aBound = []
df_aSmall = []
df_aData = []

df_aBound = df_anomalies[df_anomalies['IQR_out_of_bound'] == True]
df_aSmall = df_anomalies[(df_anomalies['IQR_to_small'] == True) & (df_anomalies['IQR_out_of_bound'] == False)]
df_aData = df_anomalies[(df_anomalies['not_enough_data'] == True) & (df_anomalies['IQR_out_of_bound'] == False) & (df_anomalies['IQR_to_small'] == False)]


In [24]:
#df = pd.merge(df_sensors, df_anomalies, on='detid', how='left')
df_anomBound = pd.merge(df_aBound, df_sensors, on='detid', how='left')
df_anomSmall = pd.merge(df_aSmall, df_sensors, on='detid', how='left')
df_anomData = pd.merge(df_aData, df_sensors, on='detid', how='left')


In [26]:
import folium

def plot_sensors_as_points(df1_blue, df2_red, df3_green, lat_col='lat', long_col='long', sensorid_col='detid', trafficIndex_col=None, city_center=(51.5074, -0.1278), zoom_start=12):
    """
    Plot the sensor locations as points (CircleMarkers) on a map of London.
    
    Args:
    - df: DataFrame containing sensor data with latitude and longitude columns.
    - lat_col: Name of the column containing latitude values (default: 'lat').
    - long_col: Name of the column containing longitude values (default: 'long').
    - sensorid_col: Name of the column containing sensor IDs (default: 'detid').
    - trafficIndex_col: Optional. Name of the column containing traffic index values (for popup display).
    - city_center: Tuple of (latitude, longitude) for the center of the map (default is central London).
    - zoom_start: Initial zoom level for the map (default is 12).
    
    Output:
    - Folium map with sensor locations plotted as CircleMarkers (points).
    """
    # Create a Folium map centered around London
    m = folium.Map(location=city_center, zoom_start=zoom_start)
        
    # Plot each sensor as a point on the map using CircleMarker
    for _, row in df1_blue.iterrows():
        # Popup text with sensor id and optional traffic index
        popup_text = f"Sensor ID: {row[sensorid_col]}"
        if trafficIndex_col:
            popup_text += f"<br>Traffic Index: {row[trafficIndex_col]}"
        
        # Add a CircleMarker for each sensor
        folium.CircleMarker(
            location=[row[lat_col], row[long_col]],
            radius=3,  # Size of the circle (points)
            color='blue',  # Outline color
            fill=True,
            fill_color='blue',  # Fill color of the point
            fill_opacity=0.7,
            popup=popup_text
        ).add_to(m)
        
    for _, row in df2_red.iterrows():
        # Popup text with sensor id and optional traffic index
        popup_text = f"Sensor ID: {row[sensorid_col]}"
        if trafficIndex_col:
            popup_text += f"<br>Traffic Index: {row[trafficIndex_col]}"
        
        # Add a CircleMarker for each sensor
        folium.CircleMarker(
            location=[row[lat_col], row[long_col]],
            radius=3,  # Size of the circle (points)
            color='red',  # Outline color
            fill=True,
            fill_color='red',  # Fill color of the point
            fill_opacity=0.7,
            popup=popup_text
        ).add_to(m)
        
        for _, row in df3_green.iterrows():
            # Popup text with sensor id and optional traffic index
            popup_text = f"Sensor ID: {row[sensorid_col]}"
            if trafficIndex_col:
                popup_text += f"<br>Traffic Index: {row[trafficIndex_col]}"
            
            # Add a CircleMarker for each sensor
            folium.CircleMarker(
                location=[row[lat_col], row[long_col]],
                radius=3,  # Size of the circle (points)
                color='green',  # Outline color
                fill=True,
                fill_color='green',  # Fill color of the point
                fill_opacity=0.7,
                popup=popup_text
            ).add_to(m)

    return m


Der Plot zeigt alle Anomalien in Rot und die noch bestehenden Sensoren in Blau

In [25]:
#anomalies
plot = plot_sensors_as_points(df_anomBound, df_anomSmall, df_anomData, lat_col='lat', long_col='long', sensorid_col='detid')
#plot.save(r"C:\Users\rueed\OneDrive\HSLU\3 Semester\DSPRO 1\HSLU_DSPRO1_TrafficStatus\notebooks\grid\anomalies\Anomaliesmap_anomalies_31_10_2024.html")
plot

NameError: name 'plot_sensors_as_points' is not defined